# Yolov5 Model Egg Sex Prediction
The custom dataset must be formatted correctly in order for YoloV5 to work with the ultralytics module. See reference notebook: https://github.com/roboflow/notebooks/blob/main/notebooks/train-yolov5-classification-on-custom-data.ipynb.

## Loading Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls '/content/drive/MyDrive/CS 163/egg_sex

full_256  HH19_256	    HH19_512  HH25_256_png.zip	larger_test_set
full_512  HH19_256_png.zip  HH25_256  HH25_512


In [ ]:
!unzip '/content/drive/MyDrive/CS 163/egg_sex/HH19_256_png.zip' -d ./datasets # Path to HH19 dataset
!unzip '/content/drive/MyDrive/CS 163/egg_sex/HH25_256_png.zip' -d ./datasets # Path to HH25 dataset

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


## Download Ultralytics codebase

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-383-g1435a8ee Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (8 CPUs, 51.0 GB RAM, 32.7/235.7 GB disk)


In [ ]:
from utils.downloads import attempt_download

p5 = ['n', 's', 'm', 'l', 'x']  # P5 models
cls = [f'{x}-cls' for x in p5]  # classification models

for x in cls:
    attempt_download(f'weights/yolov5{x}.pt')

100%|██████████| 4.87M/4.87M [00:00<00:00, 61.9MB/s]

100%|██████████| 10.5M/10.5M [00:00<00:00, 103MB/s]

100%|██████████| 24.9M/24.9M [00:00<00:00, 85.6MB/s]

100%|██████████| 50.9M/50.9M [00:00<00:00, 76.5MB/s]

100%|██████████| 92.0M/92.0M [00:00<00:00, 144MB/s]



In [ ]:
%cd /content/yolov5

/content/yolov5


## Model Training and Results

In [ ]:
# TRAINING
!python classify/train.py --model yolov5s-cls.pt --data HH19_256_png --lr0 0.0001 --epochs 200 --img 256

2024-11-18 09:01:32.170574: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-18 09:01:32.190588: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-18 09:01:32.196558: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
classify/train: model=yolov5s-cls.pt, data=HH19_256_png, epochs=200, batch_size=64, imgsz=256, nosave=False, cache=None, device=, workers=8, project=runs/train-cls, name=exp, exist_ok=False, pretrained=True, optimizer=Adam, lr0=0.0001, decay=5e-05, label_smoothing=0.1, cutoff=None, dropout=None, verbose=False, seed=0, local_rank=-1
github: up to date with https:/

In [ ]:
# VALIDATION SET
!python classify/val.py --weights runs/train-cls/exp8/weights/best.pt --data /content/datasets/HH25_256_png/

classify/val: data=/content/datasets/HH25_256_png/, weights=['runs/train-cls/exp8/weights/best.pt'], batch_size=128, imgsz=224, device=, workers=8, verbose=True, project=runs/val-cls, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-383-g1435a8ee Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 117 layers, 4169250 parameters, 0 gradients, 10.4 GFLOPs
validating:   0% 0/2 [00:00<?, ?it/s]/content/yolov5/classify/val.py:111: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=device.type != "cpu"):
validating: 100% 2/2 [00:01<00:00,  1.55it/s]
                   Class      Images    top1_acc    top5_acc
                     all         162       0.525           1
                  female          74       0.419           1
                    male          88       0.614           1
Speed: 0.1ms pre-process, 3.8ms inferen

In [ ]:
# TEST SET
!python classify/predict.py --weights runs/train-cls/exp5/weights/best.pt --source /content/datasets/HH25_256_png/test/female/testing_image_103.png

classify/predict: weights=['runs/train-cls/exp/weights/best.pt'], source=/content/datasets/HH19_256_dataset/test/female/testing_image_103.png, data=data/coco128.yaml, imgsz=[224, 224], device=, view_img=False, save_txt=False, nosave=False, augment=False, visualize=False, update=False, project=runs/predict-cls, name=exp, exist_ok=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-383-g1435a8ee Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 117 layers, 4169250 parameters, 0 gradients, 10.4 GFLOPs
image 1/1 /content/datasets/HH19_256_dataset/test/female/testing_image_103.png: 224x224 female 0.61, male 0.39, 2.9ms
Speed: 0.3ms pre-process, 2.9ms inference, 29.8ms NMS per image at shape (1, 3, 224, 224)
Results saved to runs/predict-cls/exp8
